# Gerenciamento de Parâmetros

Depois de escolher uma arquitetura
e definir nossos hiperparâmetros,
passamos para o ciclo de treinamento,
onde nosso objetivo é encontrar valores de parâmetro
que minimizam nossa função de perda.
Após o treinamento, precisaremos desses parâmetros
para fazer previsões futuras.
Além disso, às vezes desejamos
para extrair os parâmetros
seja para reutilizá-los em algum outro contexto,
para salvar nosso modelo em disco para que
pode ser executado em outro *software*,
ou para exame na esperança de
ganhar compreensão científica.

Na maioria das vezes, seremos capazes de
ignorar os detalhes essenciais
de como os parâmetros são declarados
e manipulado, contando com estruturas de *Deep Learning*
para fazer o trabalho pesado.
No entanto, quando nos afastamos de
arquiteturas empilhadas com camadas padrão,
às vezes precisaremos 
declarar e manipular parâmetros.
Nesta seção, cobrimos o seguinte:

* Parâmetros de acesso para depuração, diagnóstico e visualizações.
* Inicialização de parâmetros.
* Parâmetros de compartilhamento em diferentes componentes do modelo.

Começamos nos concentrando em um MLP com uma camada oculta.


In [1]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.08388045],
       [-0.040921  ]], dtype=float32)>

## Acesso a Parâmetros

Vamos começar explicando como acessar os parâmetros
dos modelos que você já conhece.
Quando um modelo é definido por meio da classe `Sequential`,
podemos primeiro acessar qualquer camada indexando
no modelo como se fosse uma lista.
Os parâmetros de cada camada são convenientemente
localizado em seu atributo.
Podemos inspecionar os parâmetros da segunda camada totalmente conectada da seguinte maneira.


In [2]:
print(net.layers[2].weights)

[<tf.Variable 'dense_1/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.8849803 ],
       [ 0.28838134],
       [ 0.8427515 ],
       [ 0.29679787]], dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


A saída nos diz algumas coisas importantes.
Primeiro, esta camada totalmente conectada
contém dois parâmetros,
correspondendo aos
pesos e vieses, respectivamente.
Ambos são armazenados como *floats* de precisão simples (float32).
Observe que os nomes dos parâmetros
nos permitem identificar de forma única
parâmetros de cada camada,
mesmo em uma rede contendo centenas de camadas.


### Parâmetros Direcionados

Observe que cada parâmetro é representado
como uma instância da classe de parâmetro.
Para fazer algo útil com os parâmetros,
primeiro precisamos acessar os valores numéricos subjacentes.
Existem várias maneiras de fazer isso.
Alguns são mais simples, enquanto outros são mais gerais.
O código a seguir extrai o viés
da segunda camada de rede neural, que retorna uma instância de classe de parâmetro, e
acessa posteriormente o valor desse parâmetro.


In [3]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


### Todos os Parâmetros de Uma Vez

Quando precisamos realizar operações em todos os parâmetros,
acessá-los um por um pode se tornar tedioso.
A situação pode ficar especialmente complicada
quando trabalhamos com blocos mais complexos (por exemplo, blocos aninhados),
uma vez que precisaríamos voltar recursivamente
através de toda a árvore para extrair
parâmetros de cada sub-bloco. Abaixo, demonstramos como acessar os parâmetros da primeira camada totalmente conectada versus acessar todas as camadas.


In [4]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.00696713,  0.06294048, -0.34609908,  0.28365868],
       [ 0.05357468, -0.3715149 , -0.21233797,  0.5667568 ],
       [ 0.75887805, -0.24776524,  0.58249944,  0.15801603],
       [-0.309555  , -0.46662605,  0.3352285 , -0.7561607 ]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.00696713,  0.06294048, -0.34609908,  0.28365868],
       [ 0.05357468, -0.3715149 , -0.21233797,  0.5667568 ],
       [ 0.75887805, -0.24776524,  0.58249944,  0.15801603],
       [-0.309555  , -0.46662605,  0.3352285 , -0.7561607 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[-0.8849803 ],
       [ 0.28838134],
       [ 0.8427515 ],
       [ 0.29679787]], dtype=float32), array([0.], dtype=float32)]


Isso nos fornece outra maneira de acessar os parâmetros da rede como segue.


In [5]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

### Coletando Parâmetros de Blocos Aninhados

Vamos ver como funcionam as convenções de nomenclatura de parâmetros
se aninharmos vários blocos uns dentro dos outros.
Para isso, primeiro definimos uma função que produz blocos
(uma fábrica de blocos, por assim dizer) e então
combine-os dentro de blocos ainda maiores.


In [6]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.04170988],
       [-0.21115464]], dtype=float32)>

Agora que projetamos a rede,
vamos ver como está organizado.


In [7]:
print(rgnet.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (2, 4)                    80        
_________________________________________________________________
dense_6 (Dense)              (2, 1)                    5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


Uma vez que as camadas são aninhadas hierarquicamente,
também podemos acessá-los como se
indexação por meio de listas aninhadas.
Por exemplo, podemos acessar o primeiro bloco principal,
dentro dele o segundo sub-bloco,
e dentro disso o viés da primeira camada,
com o seguinte.


In [8]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'sequential_2/block-1/dense_3/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

## Inicialização de Parâmetros

Agora que sabemos como acessar os parâmetros,
vamos ver como inicializá-los corretamente.
Discutimos a necessidade de inicialização adequada em :numref:`sec_numerical_stability`.
A estrutura de *Deep Learning* fornece inicializações aleatórias padrão para suas camadas.
No entanto, muitas vezes queremos inicializar nossos pesos
de acordo com vários outros protocolos. A estrutura fornece mais comumente
protocolos usados e também permite criar um inicializador personalizado.


Por padrão, Keras inicializa matrizes de ponderação uniformemente, tirando de um intervalo que é calculado de acordo com a dimensão de entrada e saída, e os parâmetros de polarização são todos definidos como zero.
O TensorFlow oferece uma variedade de métodos de inicialização no módulo raiz e no módulo `keras.initializers`.


### Inicialização *Built-in* 

Vamos começar chamando inicializadores integrados.
O código abaixo inicializa todos os parâmetros de peso
como variáveis aleatórias gaussianas
com desvio padrão de 0,01, enquanto os parâmetros de polarização são zerados.


In [9]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_7/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.00880642,  0.00698964, -0.00339042, -0.01238017],
        [-0.00936008,  0.00712868,  0.0120426 ,  0.01030472],
        [-0.00444559,  0.01566276, -0.00378464, -0.00585882],
        [-0.00496555,  0.00707446, -0.0006112 ,  0.00711826]],
       dtype=float32)>,
 <tf.Variable 'dense_7/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

Também podemos inicializar todos os parâmetros
a um determinado valor constante (digamos, 1).


In [10]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

Também podemos aplicar inicializadores diferentes para certos blocos.
Por exemplo, abaixo inicializamos a primeira camada
com o inicializador Xavier
e inicializar a segunda camada
para um valor constante de 42.


In [11]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(1)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.2608738 , -0.22718716,  0.11381751, -0.08536631],
       [-0.32224733,  0.3555506 ,  0.52300423, -0.6358117 ],
       [ 0.52879506,  0.32641047, -0.37077838, -0.72854525],
       [-0.29767555, -0.3362043 , -0.71178615, -0.1830396 ]],
      dtype=float32)>
<tf.Variable 'dense_12/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>


### Inicialização Customizada

Às vezes, os métodos de inicialização de que precisamos
não são fornecidos pela estrutura de *Deep Learning*.
No exemplo abaixo, definimos um inicializador
para qualquer parâmetro de peso $w$ usando a seguinte distribuição estranha:

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


Aqui nós definimos uma subclasse de `Initializer` e implementamos o`__call__`
função que retorna um tensor desejado de acordo com a forma e o tipo de dados.


In [12]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 5.46612  , -0.       ,  8.619307 ,  0.       ],
       [ 7.9447002,  9.824312 ,  6.20899  , -6.143732 ],
       [ 8.9025135, -6.1618114, -6.355505 , -6.9544387],
       [ 0.       , -0.       ,  5.6121492, -5.8810377]], dtype=float32)>


Observe que sempre temos a opção
de definir parâmetros diretamente.


In [13]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  1.       ,  9.619307 ,  1.       ],
       [ 8.9447   , 10.824312 ,  7.20899  , -5.143732 ],
       [ 9.9025135, -5.1618114, -5.355505 , -5.9544387],
       [ 1.       ,  1.       ,  6.6121492, -4.8810377]], dtype=float32)>

## Parâmetros *Tied*


Frequentemente, queremos compartilhar parâmetros em várias camadas.
Vamos ver como fazer isso com elegância.
A seguir, alocamos uma camada densa
e usar seus parâmetros especificamente
para definir os de outra camada.


In [14]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Checando se os parâmetros são diferentes
print(len(net.layers) == 3)

True


## Sumário

* Temos várias maneiras de acessar, inicializar e vincular os parâmetros do modelo.
* Podemos usar inicialização personalizada.


## Exercícios

1. Use o modelo `FancyMLP` definido em :numref:`sec_model_construction` e acesse os parâmetros das várias camadas.
1. Observe o documento do módulo de inicialização para explorar diferentes inicializadores.
1. Construa um MLP contendo uma camada de parâmetros compartilhados e treine-o. Durante o processo de treinamento, observe os parâmetros do modelo e gradientes de cada camada.
1. Por que compartilhar parâmetros é uma boa ideia?


[Discussão](https://discuss.d2l.ai/t/269)


<!--stackedit_data:
eyJoaXN0b3J5IjpbMjI3MjUyNjYwLC03MTk0NTg0ODEsMjUwMT
g4NTk3LC0xNTkzNzg3NzI4LDE2NDI5Nzg1MDFdfQ==
-->
